### Import the needed modules

In [ ]:
import numpy as np
import sep

### Set up for reading and displaying test images

In [ ]:
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
from matplotlib import rcParams

%matplotlib inline

rcParams["figure.figsize"] = [10.0, 8.0]

### Image is read into standard 2-d numpy array

In [ ]:
#https://docs.astropy.org/en/stable/io/fits/index.html
HDU = fits.open("Documents/Python Code-ASTR/f105_band_image.fits")
data = HDU[0].data

### Show the image

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation = "nearest", cmap = "gray", vmin = m - s, vmax = m + s, origin = "lower")
plt.colorbar();

### Measure the spatially varying of the image backround

In [ ]:
data = data.byteswap().newbyteorder()
bkgnd = sep.Background(data)

### Get the global mean and noise of the image background

In [ ]:
print(bkgnd.globalback)
print(bkgnd.globalrms)

### Evaluate the  backround as a 2-d Array

In [ ]:
bkgnd_image = bkgnd.back()

### Show the image backround

In [ ]:
plt.imshow(bkgnd_image, interpolation = "nearest", cmap = "gray", origin = "lower")
plt.colorbar();

### Evaluate the backround noise as a 2-d Array

In [ ]:
bkgnd_rms = bkgnd.rms()

### Display this backround noise and save to a pdf

In [ ]:
plt.imshow(bkgnd_rms, interpolation = "nearest", cmap = "gray", origin = "lower")
plt.colorbar();

### subtract the backround from the original data

In [ ]:
data_sub = data - bkgnd

### Create the objects

In [ ]:
objs = sep.extract(data_sub, 1.5, err=bkgnd.globalrms)

### Determine how many objects are present

In [ ]:
len(objs)

### Plot the original minus brackround, an ellipse around each detected object

In [ ]:
from matplotlib.patches import Ellipse


fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation = "nearest", cmap = "gray",
               vmin = m - s, vmax = m + s, origin = "lower")

for i in range(len(objs)):
    e = Ellipse(xy = (objs["x"][i], objs["y"][i]),
                width = 6*objs["a"][i],
                height = 6*objs["b"][i],
                angle = objs["theta"][i] * 180.0 / np.pi)
    e.set_facecolor("none")
    e.set_edgecolor("red")
    ax.add_artist(e)

In [ ]:
objs.dtype.names

### Calculate the aperture photometry at the object locations

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objs["x"], objs["y"],
                                     3.0, err = bkgnd.globalrms, gain = 1.0)

### Show the results of the first 10 objects

In [ ]:
for i in range(10):
    print("Object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### Create a histogram of the fluxes

In [ ]:
x = [0.031282, 0.031018, -0.024388, 0.001947, 0.012457, -0.011228, 0.029368, -0.009126, 0.048023, 0.027840]
plt.hist(x, 10)
plt.show()